In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn.datasets
import pandas as pd

from pathlib import Path
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from autorank import autorank, create_report

# Render matplotlib plots in the notebook
%matplotlib inline

In [2]:
PATH_CAE = Path('caesarian.csv')
CAE = pd.read_csv(PATH_CAE)
#CAE

In [3]:
PATH_ARR = Path('arrhythmia.csv')
ARR = pd.read_csv(PATH_ARR)
for col in ARR.columns:
    ARR[col] = pd.to_numeric(ARR[col],errors='coerce')
    ARR = ARR.replace(np.nan, ARR[col].mean(), regex=True)
    ARR[col] = ARR[col].astype(int)
#ARR.iloc[:, 14]

In [4]:
PATH_WEB = Path('website-phishing.csv')
WEB = pd.read_csv(PATH_WEB)
WEB.columns = WEB.columns.map(str.strip)
#WEB

## Cross Validation on Caesarian data

In [5]:
y = CAE['class'].to_numpy()
X = CAE.drop(['class'], axis=1).to_numpy()

print(X.shape, y.shape)

(80, 5) (80,)


In [6]:
models = []
models.append(('Decision Tree', DecisionTreeClassifier())) 
models.append(('Random Forest', RandomForestClassifier(n_estimators=10))) 
models.append(('Bagging', BaggingClassifier(n_estimators=10)))
models.append(('Ada Boost', AdaBoostClassifier(n_estimators=10)))
models.append(('Gradient Boost', GradientBoostingClassifier(n_estimators=10)))

lst = []
algo = list(list(zip(*models))[0])
results = pd.DataFrame(columns=algo)
for name, model in models:
    kfold = RepeatedKFold(n_splits=10, n_repeats = 100 , random_state = 42)
    scores = cross_val_score(model, X, y, cv=kfold)
    spl = np.split(scores,100)
    for i, run in enumerate(spl):
        run_mean = np.mean(run)
        run_std = np.std(run)
        #print(f"Test run accuracy for {name} is: {run_mean:.3f}  {run_std:.3f}")
        results.at[i, name] = run_mean
    mean = np.mean(scores)
    std = np.std(scores)
    #print(scores)
    #print(len(scores))
    #print(spl)
    print(f"Mean test accuracy for {name} after 100 runs on caesarian dataset: {mean:.3f} with std of {std:.3f}")
    lst.append(mean)
#print(lst)
print(results)

    

Mean test accuracy for Decision Tree after 100 runs on caesarian dataset: 0.523 with std of 0.172
Mean test accuracy for Random Forest after 100 runs on caesarian dataset: 0.544 with std of 0.170
Mean test accuracy for Bagging after 100 runs on caesarian dataset: 0.528 with std of 0.171
Mean test accuracy for Ada Boost after 100 runs on caesarian dataset: 0.612 with std of 0.162
Mean test accuracy for Gradient Boost after 100 runs on caesarian dataset: 0.606 with std of 0.168
   Decision Tree Random Forest Bagging Ada Boost Gradient Boost
0         0.5125          0.55   0.525    0.6375            0.6
1         0.4875        0.5625  0.4875       0.6         0.5875
2         0.5125        0.5375   0.525     0.575            0.6
3         0.5125         0.575   0.525    0.6125          0.625
4            0.6           0.6  0.4875      0.65         0.6625
..           ...           ...     ...       ...            ...
95        0.5125          0.45  0.4875    0.5875         0.5875
96     

### Auto Rank data from running algorithm on Caesarian 

In [7]:
results= results.astype(float, errors = 'raise')
result = autorank(results)
print(result)
create_report(result)

RankResult(rankdf=
                meanrank      mean       std  ci_lower  ci_upper effect_size  \
Ada Boost          1.555  0.611875  0.024770  0.605369  0.618381         0.0   
Gradient Boost     1.735  0.605500  0.036641  0.595877  0.615123    0.203844   
Random Forest      3.470  0.544500  0.042615  0.533308  0.555692    1.933068   
Bagging            4.010  0.528000  0.038522  0.517882  0.538118    2.589967   
Decision Tree      4.230  0.523000  0.036050  0.513532  0.532468    2.873537   

                 magnitude  
Ada Boost       negligible  
Gradient Boost       small  
Random Forest        large  
Bagging              large  
Decision Tree        large  
pvalue=3.6137582352592976e-56
cd=0.6099436402016305
omnibus=friedman
posthoc=nemenyi
all_normal=True
pvals_shapiro=[0.16704486310482025, 0.012622312642633915, 0.35619452595710754, 0.021136658266186714, 0.07734933495521545]
homoscedastic=False
pval_homogeneity=7.51383973885904e-06
homogeneity_test=bartlett
alpha=0.05
alpha_no

## Cross Validation on Phishing data

In [8]:
y = WEB['Class'].to_numpy()
X = WEB.drop(['Class'], axis=1).to_numpy()

print(X.shape, y.shape)

(11055, 30) (11055,)


In [9]:
models = []
models.append(('Decision Tree', DecisionTreeClassifier())) 
models.append(('Random Forest', RandomForestClassifier(n_estimators=10))) 
models.append(('Bagging', BaggingClassifier(n_estimators=10)))
models.append(('Ada Boost', AdaBoostClassifier(n_estimators=10)))
models.append(('Gradient Boost', GradientBoostingClassifier(n_estimators=10)))

lst = []
algo = list(list(zip(*models))[0])
results = pd.DataFrame(columns=algo)
for name, model in models:
    kfold = RepeatedKFold(n_splits=10, n_repeats = 100 , random_state = 42)
    scores = cross_val_score(model, X, y, cv=kfold)
    spl = np.split(scores,100)
    for i, run in enumerate(spl):
        run_mean = np.mean(run)
        run_std = np.std(run)
        #print(f"Test run accuracy for {name} is: {run_mean:.3f}  {run_std:.3f}")
        results.at[i, name] = run_mean
    mean = np.mean(scores)
    std = np.std(scores)
    #print(len(scores))
    print(f"Mean test accuracy for {name} after 100 runs on phishing dataset: {mean:.3f} with std of {std:.3f}")
    lst.append(mean)
#print(lst)
print(results)

Mean test accuracy for Decision Tree after 100 runs on phishing dataset: 0.965 with std of 0.005
Mean test accuracy for Random Forest after 100 runs on phishing dataset: 0.971 with std of 0.005
Mean test accuracy for Bagging after 100 runs on phishing dataset: 0.970 with std of 0.005
Mean test accuracy for Ada Boost after 100 runs on phishing dataset: 0.929 with std of 0.007
Mean test accuracy for Gradient Boost after 100 runs on phishing dataset: 0.917 with std of 0.008
   Decision Tree Random Forest   Bagging Ada Boost Gradient Boost
0       0.963184      0.970511   0.97042  0.928811       0.917232
1        0.96644      0.972229   0.97006  0.929173       0.917231
2       0.964722      0.971416  0.968431  0.929173       0.917049
3       0.964722      0.970149   0.97033  0.929534       0.916778
4       0.963546      0.972049  0.969154  0.929443       0.917051
..           ...           ...       ...       ...            ...
95       0.96436      0.970782  0.968521   0.92872       0.917

### Auto Rank data from running algorithm on Phishing 

In [10]:
results= results.astype(float, errors = 'raise')
result = autorank(results)
print(result)
create_report(result)

RankResult(rankdf=
                meanrank    median       mad  ci_lower  ci_upper effect_size  \
Random Forest       1.05  0.971234  0.001073  0.970873  0.972047         0.0   
Bagging             1.95  0.969653  0.001005  0.969245   0.97006    1.521865   
Decision Tree       3.00  0.965264   0.00134  0.964722  0.965989    4.919494   
Ada Boost           4.00  0.929218  0.000338   0.92908  0.929441   52.825585   
Gradient Boost      5.00  0.917140   0.00013  0.917052  0.917142   70.794976   

                 magnitude  
Random Forest   negligible  
Bagging              large  
Decision Tree        large  
Ada Boost            large  
Gradient Boost       large  
pvalue=1.8421898918723992e-84
cd=0.6099436402016305
omnibus=friedman
posthoc=nemenyi
all_normal=False
pvals_shapiro=[0.4343690574169159, 0.3878830671310425, 0.6997711062431335, 0.13431629538536072, 9.768938724619147e-08]
homoscedastic=False
pval_homogeneity=3.6978656331873236e-39
homogeneity_test=levene
alpha=0.05
alpha_norm

## Cross validation on Arrythmia data

In [11]:
y = ARR['class'].to_numpy()
X = ARR.drop(['class'], axis=1).to_numpy()

print(X.shape, y.shape)

(452, 279) (452,)


In [12]:
models = []
models.append(('Decision Tree', DecisionTreeClassifier())) 
models.append(('Random Forest', RandomForestClassifier(n_estimators=10))) 
models.append(('Bagging', BaggingClassifier(n_estimators=10)))
models.append(('Ada Boost', AdaBoostClassifier(n_estimators=10)))
models.append(('Gradient Boost', GradientBoostingClassifier(n_estimators=10)))

lst = []
algo = list(list(zip(*models))[0])
results = pd.DataFrame(columns=algo)
for name, model in models:
    kfold = RepeatedKFold(n_splits=10, n_repeats = 100 , random_state = 42)
    scores = cross_val_score(model, X, y, cv=kfold)
    spl = np.split(scores,100)
    for i, run in enumerate(spl):
        run_mean = np.mean(run)
        run_std = np.std(run)
        #print(f"Test run accuracy for {name} is: {run_mean:.3f}  {run_std:.3f}")
        results.at[i, name] = run_mean
    mean = np.mean(scores)
    std = np.std(scores)
    #print(len(scores))
    print(f"Mean test accuracy for {name} after 100 runs on arrhythmia dataset: {mean:.3f} with std of {std:.3f}")
    lst.append(mean)
#print(lst)
print(results)

Mean test accuracy for Decision Tree after 100 runs on arrhythmia dataset: 0.619 with std of 0.067
Mean test accuracy for Random Forest after 100 runs on arrhythmia dataset: 0.688 with std of 0.064
Mean test accuracy for Bagging after 100 runs on arrhythmia dataset: 0.726 with std of 0.065
Mean test accuracy for Ada Boost after 100 runs on arrhythmia dataset: 0.610 with std of 0.069
Mean test accuracy for Gradient Boost after 100 runs on arrhythmia dataset: 0.703 with std of 0.065
   Decision Tree Random Forest   Bagging Ada Boost Gradient Boost
0       0.654831      0.701401  0.719082  0.617391       0.736908
1       0.608599      0.681159      0.73  0.621836       0.692464
2       0.628213      0.698889  0.734396  0.617053       0.716667
3       0.619614      0.672947  0.703768  0.613043       0.677005
4        0.62599      0.687923   0.72314  0.601643        0.71657
..           ...           ...       ...       ...            ...
95      0.619614      0.668309  0.723285  0.619469  

### Auto Rank data from running algorithm on Arrythmia

In [13]:
results= results.astype(float, errors = 'raise')
result = autorank(results)
print(result)
create_report(result)

RankResult(rankdf=
                meanrank    median       mad  ci_lower  ci_upper effect_size  \
Bagging            1.130  0.725676   0.01035  0.721063  0.732222         0.0   
Gradient Boost     2.045  0.702512  0.011388  0.697053  0.706135    2.128829   
Random Forest      2.825  0.687850  0.012641  0.683671  0.692657    3.274277   
Decision Tree      4.300  0.619589  0.016187  0.610966  0.626232    7.808886   
Ada Boost          4.700  0.610725  0.003653  0.608406   0.61285   14.812082   

                 magnitude  
Bagging         negligible  
Gradient Boost       large  
Random Forest        large  
Decision Tree        large  
Ada Boost            large  
pvalue=7.530518001386896e-77
cd=0.6099436402016305
omnibus=friedman
posthoc=nemenyi
all_normal=False
pvals_shapiro=[0.5723743438720703, 0.011801826767623425, 0.7937547564506531, 0.0014996429672464728, 0.06660961359739304]
homoscedastic=False
pval_homogeneity=2.8441753164723372e-12
homogeneity_test=levene
alpha=0.05
alpha_nor